In [49]:
import pandas as pd

import matplotlib.pyplot as plt

import matplotlib.ticker as mtick

import matplotlib.pyplot as plt

import matplotlib.dates as mdates

import matplotlib.cm as cm

import numpy as np

In [50]:
attrition = pd.read_csv(r"C:\Users\Student\Desktop\Shivan\Portfolio Projects\Project 2 - Protea Capital Partners\Data\Raw Data\attrition.csv", sep = ",")

compensation = pd.read_csv(r"C:\Users\Student\Desktop\Shivan\Portfolio Projects\Project 2 - Protea Capital Partners\Data\Raw Data\compensation.csv", sep = ",")

employees = pd.read_csv(r"C:\Users\Student\Desktop\Shivan\Portfolio Projects\Project 2 - Protea Capital Partners\Data\Raw Data\employees.csv", sep = ",")

engagement = pd.read_csv(r"C:\Users\Student\Desktop\Shivan\Portfolio Projects\Project 2 - Protea Capital Partners\Data\Raw Data\engagement.csv", sep = ",")

performance = pd.read_csv(r"C:\Users\Student\Desktop\Shivan\Portfolio Projects\Project 2 - Protea Capital Partners\Data\Raw Data\performance.csv", sep = ",")

training = pd.read_csv(r"C:\Users\Student\Desktop\Shivan\Portfolio Projects\Project 2 - Protea Capital Partners\Data\Raw Data\training.csv", sep = ",")

In [51]:
attrition.dtypes # Checking datatypes

EmployeeID           int64
ExitDate            object
ExitReason          object
EstimatedCostZAR     int64
dtype: object

In [52]:
attrition.head() # Checking the first few values

,EmployeeID,ExitDate,ExitReason,EstimatedCostZAR
0,3,2024-12-09,Personal,28275
1,15,2024-03-03,Better Pay,15909
2,19,2024-07-29,Better Pay,37620
3,21,2024-06-30,Better Pay,17431
4,24,2024-12-06,Career Change,47624


In [53]:
attrition = attrition.drop_duplicates() # Removes duplicates

In [54]:
attrition.shape # Checking rows and columns

(100, 4)

In [55]:
attrition.isnull().sum() # Checking for missing values

EmployeeID          0
ExitDate            0
ExitReason          0
EstimatedCostZAR    0
dtype: int64

In [56]:
attrition.describe(include = 'all').round(2) # Getting descriptive stats and rounding to 2 decimal places.

,EmployeeID,ExitDate,ExitReason,EstimatedCostZAR
count,100.00,100,100,100.00
unique,NaN,85,6,NaN
top,NaN,2024-12-02,Better Pay,NaN
freq,NaN,3,29,NaN
mean,269.38,NaN,NaN,32136.26
std,146.16,NaN,NaN,9878.28
min,3.00,NaN,NaN,15148.00
25%,141.25,NaN,NaN,24196.50
50%,279.00,NaN,NaN,32223.50
75%,392.50,NaN,NaN,39445.50


In [57]:
attrition[(attrition['ExitDate'] < employees.set_index('EmployeeID').loc[attrition['EmployeeID'], 'HireDate'].values)]
# attrition[(attrition['ExitDate'] - References the attrition table and 'ExitDate' column. Square brackets make it a boolean mask.Brackets are closed later.
# employees.set_index('EmployeeID'), 'HireDate'] - Makes the 'EmployeeID' column the index of the employees table that we're referencing.
# .loc[attrition['EmployeeID'] - Uses the 'EmployeeID' from the attrition table to fetch the corresponding 'HireDate' from the employees table.
# .values)] - Converts it into a numpy array so it aligns with the 'ExitDate' column row by row.
# This function will pull all the data where the exit date is earlier than the hire date - revealing bad data.

,EmployeeID,ExitDate,ExitReason,EstimatedCostZAR
25,142,2024-10-20,Relocation,31150
33,189,2024-10-12,Better Pay,47031
38,240,2024-05-16,Personal,27638
53,296,2024-03-13,Better Pay,40675
62,338,2024-06-23,Career Change,26009
69,366,2024-02-13,Career Change,30321
77,409,2024-05-19,Personal,48092
95,490,2024-10-30,Workload,37391


In [58]:
# Okay, we have 8 entries where the exit date is sooner than the hire date.
# This means the data is bad because someone can't leave a company before being hired to it!
# Normally, I would speak with HR and have them provide me with accurate information so I can correct it. 
# But this is a fictional data set that I generated so I can't do that.
# So we're going to assume that the dates were erroniously switched during the data entry process.
# Now we need to write a script that will swap the exit date and hire date for these bad entries

In [59]:
attrition = attrition.copy()
employees = employees.copy()
# Making copies of the tables so we don't break anything

In [60]:
hire_dates = employees.set_index('EmployeeID').loc[attrition['EmployeeID'], 'HireDate'].values
# Extracting the hire dates for each employee ID from the employees table to make life easy

In [61]:
bad_dates = attrition['ExitDate'] < hire_dates
# Now we're pulling out all the bad dates where the exit date is earlier than the hire date
# See? 'hire_dates' made life easy

In [62]:
temp_exit = attrition.loc[bad_dates, 'ExitDate']
# Storing the bad dates temporarily

In [63]:
attrition.loc[bad_dates, 'ExitDate'] = hire_dates[bad_dates]
# Swapping the dates that match the 'bad_dates' values in the 'ExitDate' column of the attrition table with the hire dates in the bad_dates table

In [64]:
employees.loc[employees['EmployeeID'].isin(attrition.loc[bad_dates, 'EmployeeID']), 'HireDate'] = temp_exit.values
# Replacing the bad hire dates with the corresponding end dates.
# attrition.loc[bad_dates, 'EmployeeID'] - References the employee IDs with bad dates
# employees['EmployeeID'].isin(...) - Checks for the employee IDs with bad dates (works with the above)
# 'HireDate' - Works with the 'HireDate' column
# temp_exit.values - Replaces the values

In [65]:
attrition[(attrition['ExitDate'] < employees.set_index('EmployeeID').loc[attrition['EmployeeID'], 'HireDate'].values)]
# Checking for bad dates again to see if it was fixed

,EmployeeID,ExitDate,ExitReason,EstimatedCostZAR


In [66]:
# Zero entries! IT WORKED! That took HOURS to figure out! WOW!

In [67]:
attrition['ExitReason'].value_counts()
# Just counting the reasons people left. This will help us later and reveal outliers now.

ExitReason
Better Pay       29
Management       19
Personal         18
Career Change    13
Relocation       11
Workload         10
Name: count, dtype: int64

In [68]:
# Pay your people! No outliers though. That's good.

In [69]:
attrition.dtypes # Double checking that everything is okay.

EmployeeID           int64
ExitDate            object
ExitReason          object
EstimatedCostZAR     int64
dtype: object

In [70]:
attrition.head() # Another double check

,EmployeeID,ExitDate,ExitReason,EstimatedCostZAR
0,3,2024-12-09,Personal,28275
1,15,2024-03-03,Better Pay,15909
2,19,2024-07-29,Better Pay,37620
3,21,2024-06-30,Better Pay,17431
4,24,2024-12-06,Career Change,47624


In [71]:
# Data cleaning of attrition table complete. Let's move on to the compensation table.

In [72]:
compensation.dtypes

EmployeeID        int64
EffectiveDate    object
BaseSalaryZAR     int64
BonusZAR          int64
TotalCompZAR      int64
dtype: object

In [73]:
compensation.head()

,EmployeeID,EffectiveDate,BaseSalaryZAR,BonusZAR,TotalCompZAR
0,1,2025-01-01,54832,10203,65035
1,2,2025-01-01,22446,4297,26743
2,3,2025-01-01,65583,7434,73017
3,4,2025-01-01,53392,5691,59083
4,5,2025-01-01,22461,4277,26738


In [74]:
compensation = compensation.drop_duplicates() # Removing duplicates

In [75]:
compensation.isnull().sum() # Checking for missing values

EmployeeID       0
EffectiveDate    0
BaseSalaryZAR    0
BonusZAR         0
TotalCompZAR     0
dtype: int64

In [76]:
compensation.shape # Checking rows and columns

(500, 5)

In [77]:
compensation.describe(include = 'all').round(2) # Descriptive stats

,EmployeeID,EffectiveDate,BaseSalaryZAR,BonusZAR,TotalCompZAR
count,500.00,500,500.00,500.00,500.00
unique,NaN,1,NaN,NaN,NaN
top,NaN,2025-01-01,NaN,NaN,NaN
freq,NaN,500,NaN,NaN,NaN
mean,250.50,NaN,45962.02,5865.94,51827.96
std,144.48,NaN,18971.38,3273.02,21544.12
min,1.00,NaN,15138.00,895.00,16469.00
25%,125.75,NaN,29125.25,3249.75,32855.50
50%,250.50,NaN,44361.00,5167.50,49413.50
75%,375.25,NaN,63469.75,8201.00,72133.00


In [78]:
(compensation[['BaseSalaryZAR', 'BonusZAR']] <= 0).sum()
# Checking for negative values to verify data integrity

BaseSalaryZAR    0
BonusZAR         0
dtype: int64

In [79]:
compensation['EmployeeID'].duplicated().sum()
# Checking if there are duplicate employee IDs

np.int64(0)

In [80]:
# I want to do a check to see how much each person earns based on their role to identify outliers.
# I need to check the tables to find the table with the roles.

In [81]:
employees.head()

,EmployeeID,Gender,Age,HireDate,DepartmentID,RoleID,EmploymentType,Status
0,1,M,42,2022-03-04,4,1,Permanent,Active
1,2,F,53,2016-04-01,4,3,Permanent,Active
2,3,M,44,2015-04-13,3,5,Permanent,Terminated
3,4,M,54,2023-04-26,4,4,Permanent,Active
4,5,M,24,2018-01-31,7,3,Permanent,Active


In [82]:
# This table has a 'RoleID' column but that's not very descriptive of what the role is. Let's keep looking.

In [83]:
engagement.head()

,EmployeeID,SurveyDate,EngagementScore,WorkLifeBalance,JobSatisfaction
0,1,2024-12-01,51,3,1
1,2,2024-12-01,69,2,3
2,3,2024-12-01,44,1,3
3,4,2024-12-01,73,2,3
4,5,2024-12-01,61,5,3


In [84]:
performance.head()

,EmployeeID,ReviewDate,PerformanceScore,PromotionEligible,PromotionGranted
0,1,2024-12-31,3,0,0
1,2,2024-12-31,2,0,0
2,3,2024-12-31,3,0,0
3,4,2024-12-31,5,0,0
4,5,2024-12-31,2,0,0


In [85]:
training.head()

,EmployeeID,TrainingDate,TrainingType,Hours
0,1,2024-01-13,Finance,22
1,1,2024-01-24,Technical,33
2,2,2024-05-05,Soft Skills,24
3,2,2024-08-31,Compliance,34
4,2,2024-11-08,Soft Skills,34


In [86]:
# It seems like RoleID is all I have to work with. Would recommend the table be updated to include the job title, department, or level.
# I need to join the compensation table with the employees table to pull the role IDs

In [87]:
role_comp = compensation.merge( # Names the new table and starts the merge 
    employees[['EmployeeID', 'RoleID']], # Bringing in the 'employeeID' and 'RoleID' columns from the employees table
    on = 'EmployeeID', # 'EmployeeID' is our reference column
    how = 'left' # Left join

_IncompleteInputError: incomplete input (190909831.py, line 4)

In [88]:
# Oh, close the brackets... Duh!

In [89]:
role_comp = compensation.merge( # Names the new table and starts the merge 
    employees[['EmployeeID', 'RoleID']], # Bringing in the 'employeeID' and 'RoleID' columns from the employees table
    on = 'EmployeeID', # 'EmployeeID' is our reference column
    how = 'left' # Left join
)

In [90]:
role_comp.head() # Checking the table

,EmployeeID,EffectiveDate,BaseSalaryZAR,BonusZAR,TotalCompZAR,RoleID
0,1,2025-01-01,54832,10203,65035,1
1,2,2025-01-01,22446,4297,26743,3
2,3,2025-01-01,65583,7434,73017,5
3,4,2025-01-01,53392,5691,59083,4
4,5,2025-01-01,22461,4277,26738,3


In [91]:
role_comp # Working with role_comp table
    .groupby('RoleID')['BaseSalaryZar'] # Grouping by 'RoleID' and working with 'BaseSalaryZAR' columns
    .describe() # Descriptive stats
    .round(2) # Rounding to 2 decimal places. 
# Do I need brackets?

IndentationError: unexpected indent (4008174361.py, line 2)

In [92]:
# I think I need brackets...

In [93]:
role_comp ( # Working with role_comp table
    .groupby('RoleID')['BaseSalaryZar'] # Grouping by 'RoleID' and working with 'BaseSalaryZAR' columns
    .describe() # Descriptive stats
    .round(2) # Rounding to 2 decimal places. 
)

SyntaxError: invalid syntax (2020181090.py, line 2)

In [94]:
# Apparently I can't write it out like that... let's try one line

In [95]:
role_comp.groupby('RoleID')['BaseSalaryZAR'].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
RoleID,,,,,,,,
1,51.0,49330.02,19172.23,15882.0,34104.00,54832.0,65373.50,79353.0
2,58.0,41440.09,18067.27,15159.0,26759.75,37315.0,50214.25,77551.0
3,55.0,41299.78,17703.51,16589.0,26767.00,40833.0,52604.50,79815.0
4,45.0,45038.27,18373.40,15138.0,31566.00,41917.0,60505.00,73653.0
5,43.0,41238.93,19437.78,16960.0,25701.00,34225.0,55894.00,79047.0
6,46.0,52108.37,18901.81,16207.0,39037.50,54376.0,70740.00,77493.0
7,45.0,47705.87,19953.26,15340.0,29138.00,48210.0,67425.00,79400.0
8,55.0,50856.87,17826.88,18236.0,36983.00,52226.0,66477.50,78922.0
9,52.0,47436.50,20518.90,16098.0,27925.25,49514.0,66637.00,79962.0


In [96]:
# This data is very strange. There is very little variance between the roles.
# We would expect greater variance as you move up the ranks.
# There should be management staff, skilled professionals, admin staff, etc here.
# So why are there as many managers as there are support staff? Why are their salaries so close? What's going on?
# We don't know what RoleID corresponds with which role/level. We would need to check with HR and investigate the data.
# Are the salary bands really this close? This may indicate pay compression issues.

In [97]:
# Note: At this stage I am tempted to simply regenerate the dataset and fix these issues. It would make my life much easier. 
# However, in the working world we don't always get neat and tidy data sets to work with.
# So I will continue working with this messy data set so that you can see how I work under pressure. 

In [103]:
role_comp['ZScore'] = (
    role_comp['BaseSalaryZAR'] 
    - role_comp.groupby('RoleID')['BaseSalaryZAR'].transform('mean') # Determines the mean salary of each role ID
) / role_comp.groupby('RoleID')['BaseSalaryZAR'].transform('std') # Determines the standard deviation of each role ID

role_comp[np.abs(role_comp['ZScore']) > 3] # Displays employees with a std deviation greater than 3
# We're essentially looking for statistical anomalies.

,EmployeeID,EffectiveDate,BaseSalaryZAR,BonusZAR,TotalCompZAR,RoleID,ZScore


In [107]:
set(compensation['EmployeeID']) - set(employees['EmployeeID'])
# Converts the values being grabbed into sets
# Grabs the 'EmployeeID' from the compensation and employees tables
# Checks for Employee IDs in the compensation table that do not exist in the employees table
# This is flag salaries being paid to people not in the employee table, alerting us to orphan salaries

set()

In [110]:
(compensation['BonusZAR'] / compensation['BaseSalaryZAR']).describe().round(2)
# Divides the bonus by the salary in the compensation table.
# This provides us with our bonus/salary ratio
# We then get the descriptive stats of this ratio, rounded to 2 decimal places.

count    500.00
mean       0.13
std        0.04
min        0.05
25%        0.09
50%        0.13
75%        0.16
max        0.20
dtype: float64

In [111]:
# There doesnt seem to be any outliers. All numbers fall within reasonable ranges.

In [112]:
# All checks for the compensation table is done now. 
# We identified some concerning data in the compensation salary bands. We'll keep an eye on that moving forward.
# Let's clean the employees table

In [113]:
employees.dtypes

EmployeeID         int64
Gender            object
Age                int64
HireDate          object
DepartmentID       int64
RoleID             int64
EmploymentType    object
Status            object
dtype: object

In [114]:
employees.head()

,EmployeeID,Gender,Age,HireDate,DepartmentID,RoleID,EmploymentType,Status
0,1,M,42,2022-03-04,4,1,Permanent,Active
1,2,F,53,2016-04-01,4,3,Permanent,Active
2,3,M,44,2015-04-13,3,5,Permanent,Terminated
3,4,M,54,2023-04-26,4,4,Permanent,Active
4,5,M,24,2018-01-31,7,3,Permanent,Active


In [115]:
employees.shape

(500, 8)

In [116]:
employees.isnull().sum()

EmployeeID        0
Gender            0
Age               0
HireDate          0
DepartmentID      0
RoleID            0
EmploymentType    0
Status            0
dtype: int64

In [117]:
employees.describe(include = 'all').round(2)

,EmployeeID,Gender,Age,HireDate,DepartmentID,RoleID,EmploymentType,Status
count,500.00,500,500.00,500,500.00,500.00,500,500
unique,NaN,2,NaN,472,NaN,NaN,2,2
top,NaN,F,NaN,2015-11-19,NaN,NaN,Permanent,Active
freq,NaN,256,NaN,2,NaN,NaN,374,400
mean,250.50,NaN,40.99,NaN,4.04,5.45,NaN,NaN
std,144.48,NaN,11.00,NaN,2.02,2.94,NaN,NaN
min,1.00,NaN,22.00,NaN,1.00,1.00,NaN,NaN
25%,125.75,NaN,31.75,NaN,2.00,3.00,NaN,NaN
50%,250.50,NaN,42.00,NaN,4.00,5.00,NaN,NaN
75%,375.25,NaN,50.00,NaN,6.00,8.00,NaN,NaN


In [118]:
# Well looking at this made me realise that I don't need descriptive stats for this data
# The data is more qualitative than quantitative

In [119]:
employees = employees.drop_duplicates()

In [123]:
employees['EmployeeID'].duplicated().sum() # Checks for duplicated employee IDs

np.int64(0)

In [125]:
# Checks for the employees table are done. Pretty simple stuff here as it's not quantitative data. 
# Let's move on to the engagement table.

In [126]:
engagement.head()

,EmployeeID,SurveyDate,EngagementScore,WorkLifeBalance,JobSatisfaction
0,1,2024-12-01,51,3,1
1,2,2024-12-01,69,2,3
2,3,2024-12-01,44,1,3
3,4,2024-12-01,73,2,3
4,5,2024-12-01,61,5,3


In [127]:
engagement.dtypes

EmployeeID          int64
SurveyDate         object
EngagementScore     int64
WorkLifeBalance     int64
JobSatisfaction     int64
dtype: object

In [128]:
engagement.shape

(500, 5)

In [129]:
engagement.describe(include = 'all').round(2)

,EmployeeID,SurveyDate,EngagementScore,WorkLifeBalance,JobSatisfaction
count,500.00,500,500.00,500.00,500.00
unique,NaN,1,NaN,NaN,NaN
top,NaN,2024-12-01,NaN,NaN,NaN
freq,NaN,500,NaN,NaN,NaN
mean,250.50,NaN,69.22,2.93,2.88
std,144.48,NaN,16.40,1.44,1.44
min,1.00,NaN,40.00,1.00,1.00
25%,125.75,NaN,56.00,2.00,2.00
50%,250.50,NaN,68.00,3.00,3.00
75%,375.25,NaN,83.00,4.00,4.00
